In [1]:
from sqlalchemy import create_engine, text
from sqlalchemy.engine import URL
from urllib.parse import unquote

# Parametrar för anslutning
server_name = "localhost"
database_name = "ArijanBerishaDB"
connection_string = (
    f"DRIVER=ODBC Driver 18 for SQL Server;"
    f"SERVER={server_name};"
    f"DATABASE={database_name};"
    f"Trusted_Connection=yes;"
    f"TrustServerCertificate=yes"
)

# Skapa SQLAlchemy URL
url_string = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

# Debug: visa anslutningssträngen
print('Connecting to database using URL string:')
unquoted_url = unquote(str(url_string))
print(unquoted_url, '\n')

# Anslut och kör sökning
try:
    engine = create_engine(url_string)
    with engine.connect() as connection:
        print(f'Successfully connected to {database_name}!\n')

        # Fråga användaren efter söktermen
        sokterm = input("Ange manga-titel att söka efter: ")

        query = text("""
            SELECT b.ISBN13, b.Titel, b.Pris, bu.Namn AS Butik, ls.Antal
            FROM Bocker b
            JOIN LagerSaldo ls ON b.ISBN13 = ls.ISBN
            JOIN Butiker bu ON ls.ButikID = bu.ID
            WHERE b.Titel LIKE :soktitel
            ORDER BY b.Titel, bu.Namn;
        """)

        result = connection.execute(query, {"soktitel": f"%{sokterm}%"})
        rows = result.fetchall()

        if not rows:
            print("\nInga böcker hittades som matchar sökningen.")
        else:
            print("\nSökresultat:")
            for row in rows:
                print(f"{row.Titel} (ISBN: {row.ISBN13}) - {row.Pris} kr - {row.Butik}: {row.Antal} st")

except Exception as e:
    print('\nError while connecting to database or running query:')
    print(e)



Connecting to database using URL string:
mssql+pyodbc://?odbc_connect=DRIVER=ODBC+Driver+18+for+SQL+Server;SERVER=localhost;DATABASE=ArijanBerishaDB;Trusted_Connection=yes;TrustServerCertificate=yes 

Successfully connected to ArijanBerishaDB!


Sökresultat:
Attack on Titan Vol. 1 (ISBN: 9781612620244) - 105.00 kr - Comics Heaven: 6 st
Attack on Titan Vol. 2 (ISBN: 9781612620251) - 105.00 kr - Comics Heaven: 4 st
